<a href="https://colab.research.google.com/github/VijayaBhargavi198/5731Assignments/blob/master/Inclass_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium
!pip install textblob

In [103]:
# Webscrap and collect data using selenium
from selenium import webdriver
import time
import pandas as pd
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)
driver.get("https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv") 
for i in range(0,3):
  button =driver.find_element_by_xpath('//*[@id = "load-more-trigger"]').click()
  time.sleep(5)
output = list()
reviews = driver.find_elements_by_class_name("content")
for review in reviews:
  output.append(review.find_element_by_css_selector(".text.show-more__control").text)
  pd.DataFrame(output,columns = ["review"]).to_csv("User_reviews")
#create data frame for all the reviews for data cleaning
result_df= pd.DataFrame(output,columns = ["User_Review"])
#result_df

In [105]:
#Data Cleaning steps:
#1.Remove noise, such as special characters and punctuations.
#2.Remove numbers
#3.Remove stopwords by using the stopwords list
#4.Lowercase all texts
#5.Stemming
#6.Lemmatization.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
import re
stemmer=PorterStemmer()
lemmatizer = WordNetLemmatizer() 
stop_words=stopwords.words("english")
review_filter_stemming=[]
review_filter_Lemmatization=[]
final_stemmed_review=[]
final_lemmatized_review=[]
# cleaning the text data
result_df['User_Review'] = result_df['User_Review'].str.replace(r"\W", " ").str.strip()# 1.To remove special characters and punctuations
result_df['User_Review']= result_df['User_Review'].str.replace(r'\d+',"") #2.To remove Numbers
for a in result_df['User_Review']:
    splitting_words=word_tokenize(a)
    for b in splitting_words:
        if b not in stop_words: #3.Removing Stop Words
            b=b.lower() # 4.Coverting text to lower case 
            stemming=stemmer.stem(b) # 5.perfroming stemming
            review_filter_stemming.append(stemming)
            lemmatization_words=lemmatizer.lemmatize(b.lower()) # 6.Lemmatization
            review_filter_Lemmatization.append(lemmatization_words)
    final_stemmed_review.append(' '.join(review_filter_stemming))
    final_lemmatized_review.append(' '.join(review_filter_Lemmatization))
    review_filter_stemming.clear()
    review_filter_Lemmatization.clear()
result_df['Review after Stemming']=pd.DataFrame(final_stemmed_review)
result_df['Review after Lemmatization']=pd.DataFrame(final_lemmatized_review)
result_df['word_count'] = result_df["Review after Lemmatization"].apply(lambda x : len(x.split(" ")))
#result_df


In [106]:
# function definition to compute the tf idf and tf-idf values for the starting word in the text

#computeTF
def computeTF(sentence):
  words = sentence.split(" ")
  value = len(set(words))
  return words.count(words[0])/value
  pass

#compute IDF
import math 
def computeIDF(sentence):
  words = sentence.lower().split(" ")
  res = words.count(words[0])
  return math.log(len(words)/res, 10)
  pass

In [107]:
result_df["tf"] = result_df["Review after Lemmatization"].apply(lambda x : round(computeTF(x),4))
result_df["idf"] = result_df["Review after Lemmatization"].apply(lambda x : round(computeIDF(x),4))
result_df["tf-idf"] = result_df["tf"]*result_df["idf"]
#result_df

In [108]:
# syntax and structure analysis
#3.1 Parts of Speech (POS) Tagging: 
import nltk
from collections import Counter
#initialising
tags=[]
count_tag=[]
count=0
# POS tagging
for val in result_df['Review after Lemmatization']:
    word_tokens=word_tokenize(val)
    POS_tags=nltk.pos_tag(word_tokens)
    tags.append(POS_tags)
from collections import Counter
def get_pos(value,tags):
  pos_counts=[]
  for j in tags:
    counts = Counter(tag for word,tag in j)   
    pos_counts.append(counts.get(value))
  return pos_counts


In [115]:
import numpy as np
result_df["Number of nouns"] = pd.DataFrame(get_pos("NN",tags))
result_df["Number of verbs"] = pd.DataFrame(get_pos("VB",tags))
result_df["Number of predeterminers"] = pd.DataFrame(get_pos("PDT",tags))
result_df["Number of adverbs"] = pd.DataFrame(get_pos("RB",tags))
result_df["Number of cardinal digits"] = pd.DataFrame(get_pos("CD", tags))
result_df["Number of prepositions"] = pd.DataFrame(get_pos("IN",tags))
result_df["Number of Determiners"] = pd.DataFrame(get_pos("DT",tags)) 
result_df["Number of adjectives"] = pd.DataFrame(get_pos("JJ",tags)) 
result_df["Number of conjunctions"] = pd.DataFrame(get_pos("CC",tags))
result_df["Number of pronouns"] = pd.DataFrame(get_pos("PP",tags))
result_df["Number of Punctuations"] = pd.DataFrame(get_pos(":",tags))
result_df = result_df.fillna(0)
pronouns = pd.DataFrame(get_pos("PP",tags))
prepositions = pd.DataFrame(get_pos("IN",tags))
punctuations = pd.DataFrame(get_pos(":",tags))
conjunctions = pd.DataFrame(get_pos("CC",tags))
result_df["Density of pronouns"] = result_df["Number of pronouns"]/pronouns[0]
result_df["Density of prepositions"] = result_df["Number of prepositions"]/prepositions[0]
result_df["Density of punctuations"] = result_df["Number of Punctuations"]/punctuations[0]
result_df["Density of conjunctions"] = result_df["Number of conjunctions"]/conjunctions[0]
#result_df



In [116]:
final_df = result_df[['User_Review','word_count','tf','idf','tf-idf','Number of nouns','Number of adverb','Number of cardinal digits','Number of Verbs','Number of Determiners','Number of adjectives','Number of interjenctions',"Density of pronouns","Density of prepositions","Density of punctuations","Density of conjunctions"]]
final_df

,User_Review,word_count,tf,idf,tf-idf,Number of nouns,Number of adverb,Number of cardinal digits,Number of Verbs,Number of Determiners,Number of adjectives,Number of interjenctions,Density of pronouns,Density of prepositions,Density of punctuations,Density of conjunctions
0,Every once in a while a movie comes that trul...,50,0.0208,1.6990,0.035339,24.0,5.0,0.0,0.0,1.0,13.0,0,NaN,NaN,NaN,NaN
1,This is a movie that only those who have felt ...,43,0.0270,1.6335,0.044104,15.0,4.0,1.0,2.0,3.0,4.0,0,NaN,1.0,NaN,1.0
2,Truly a masterpiece The Best Hollywood film o...,86,0.0299,1.6335,0.048842,29.0,6.0,1.0,3.0,4.0,14.0,0,NaN,1.0,NaN,1.0
3,Joaquin Phoenix gives a tour de force performa...,54,0.0222,1.7324,0.038459,24.0,2.0,0.0,2.0,3.0,10.0,0,NaN,1.0,NaN,NaN
4,Most of the time movies are anticipated like t...,52,0.0213,1.7160,0.036551,18.0,4.0,0.0,1.0,2.0,11.0,0,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A mentally ill person without talents or inte...,35,0.0741,1.2430,0.092106,17.0,3.0,0.0,1.0,2.0,5.0,0,NaN,1.0,NaN,NaN
96,What a legendary character and what a legendar...,27,0.0417,1.4314,0.059689,12.0,2.0,0.0,0.0,2.0,4.0,0,NaN,1.0,NaN,NaN
97,Only audience who think what they do is alwa...,25,0.0417,1.3979,0.058292,12.0,2.0,0.0,1.0,0.0,3.0,0,NaN,1.0,NaN,1.0
98,While I enjoyed the film it felt pretty short...,75,0.0154,1.8751,0.028877,32.0,3.0,1.0,3.0,3.0,14.0,0,NaN,1.0,NaN,1.0
